In [1]:
%matplotlib widget

In [2]:
from pathlib import Path
from split_dataset import SplitDataset
from skimage import io

import numpy as np
from scipy.stats import zscore
from bouter.utilities import fast_pearson

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpecFromSubplotSpec

In [3]:
path = Path(r'C:\Users\You\Documents\test')

### plot origial

In [4]:
aligned_ds = SplitDataset(path/"aligned")
mask_all = io.imread(path/"mask.tif", plugin="tifffile")

n_bin_x, n_bin_y = 20, 10
pooled_mx = np.full((aligned_ds.shape[1], n_bin_x*2-1, n_bin_y*2-1), np.nan)
x_pooled = np.full((aligned_ds.shape[1], n_bin_x*2-1), np.nan)
y_pooled = np.full((aligned_ds.shape[1], n_bin_y*2-1), np.nan)

fig, ax = plt.subplots(3,4, figsize=(8,6), constrained_layout=True)
fig.suptitle('original each')
for plane in range(aligned_ds.shape[1]):
    aligned = aligned_ds[:,plane,:,:]
    mask = mask_all[plane,:,:]
    px_mask = np.argwhere(mask>=1)
    px_str = [str(x0)+str(y0) for x0, y0 in px_mask]
    
    x_loc = np.unique(px_mask[:,0])
    y_loc = np.unique(px_mask[:,1])
    x_bins = np.digitize(x_loc, np.linspace(x_loc.min(), x_loc.max(), n_bin_x)) - 1
    y_bins = np.digitize(y_loc, np.linspace(y_loc.min(), y_loc.max(), n_bin_y)) - 1
    
    traces = np.full((aligned.shape[0], n_bin_x, n_bin_y), np.nan)
    for x in range(n_bin_x):
        for y in range(n_bin_y):
            px_here = np.array([(x0, y0) for x0 in x_loc[x_bins==x] for y0 in y_loc[y_bins==y]])
            px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
            px_here = px_here[np.isin(px_here_str, px_str)]
            if len(px_here) > 0:
                traces[:, x, y] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    traces = zscore(traces, axis=0, nan_policy='omit')
            
    mx = np.full((n_bin_x*n_bin_y, n_bin_x*2-1, n_bin_y*2-1), np.nan)
    for x0 in range(n_bin_x):
        for y0 in range(n_bin_y):
            this = traces[:, x0, y0]
            if not np.isnan(this).all():
                for x in range(n_bin_x):
                    for y in range(n_bin_y):
                        that = traces[:, x, y]
                        if not np.isnan(that).all():
                            mx[x0*n_bin_y+y0, x-x0+n_bin_x-1, y-y0+n_bin_y-1] = fast_pearson(this, that)
    mx = np.nanmean(mx, axis=0)
    pooled_mx[plane,:,:] = mx
    
    x_traces = np.full((aligned.shape[0], n_bin_x), np.nan)
    for x in range(n_bin_x):
        px_here = np.array([(x0, y0) for x0 in x_loc[x_bins==x] for y0 in y_loc])
        px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
        px_here = px_here[np.isin(px_here_str, px_str)]
        if len(px_here) > 0:
            x_traces[:, x] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    x_traces = zscore(x_traces, axis=0, nan_policy='omit')
    x_mx = np.full((n_bin_x, n_bin_x*2-1), np.nan)
    for x0 in range(n_bin_x):
        this = x_traces[:, x0]
        if not np.isnan(this).all():
            for x in range(n_bin_x):
                that = x_traces[:, x]
                if not np.isnan(that).all():
                    x_mx[x0, x-x0+n_bin_x-1] = fast_pearson(this, that)
    x_mx = np.nanmean(x_mx, axis=0)
    x_pooled[plane,:] = x_mx

    y_traces = np.full((aligned.shape[0], n_bin_y), np.nan)
    for y in range(n_bin_y):
        px_here = np.array([(x0, y0) for x0 in x_loc for y0 in y_loc[y_bins==y]])
        px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
        px_here = px_here[np.isin(px_here_str, px_str)]
        if len(px_here) > 0:
            y_traces[:, y] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    y_traces = zscore(y_traces, axis=0, nan_policy='omit')
    y_mx = np.full((n_bin_y, n_bin_y*2-1), np.nan)
    for y0 in range(n_bin_y):
        this = y_traces[:, y0]
        if not np.isnan(this).all():
            for y in range(n_bin_y):
                that = y_traces[:, y]
                if not np.isnan(that).all():
                    y_mx[y0, y-y0+n_bin_y-1] = fast_pearson(this, that)
    y_mx = np.nanmean(y_mx, axis=0)
    y_pooled[plane,:] = y_mx
    
    ax.flat[plane].matshow(mx.T, vmin=-0.6, vmax=0.6, cmap='coolwarm')
    
ax.flat[-1].matshow(np.nanmean(pooled_mx, axis=0).T, vmin=-0.6, vmax=0.6, cmap='coolwarm')
ax.flat[-1].set(title='pooled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\You\miniconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
C:\Users\You\miniconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: Mean of empty slice


[Text(0.5, 1.0, 'pooled')]

In [5]:
fig = plt.figure(figsize=(6,4), tight_layout=True)
fig.suptitle("original")
gs = fig.add_gridspec(2, 2, width_ratios=[3,10], height_ratios=[3,10], wspace=0, hspace=0)

ax = fig.add_subplot(gs[1,1])
ax.matshow(np.nanmean(pooled_mx, axis=0).T, vmin=-0.5, vmax=0.5, cmap='coolwarm')
ax.yaxis.set_ticks_position('right')
ax.xaxis.set_ticks_position('bottom')
ax.set(
    xticks=range(2*n_bin_x-1), xticklabels=np.linspace(-n_bin_x+1, n_bin_x-1, 2*n_bin_x-1, dtype='int'),
    yticks=range(2*n_bin_y-1), yticklabels=np.linspace(-n_bin_y+1, n_bin_y-1, 2*n_bin_y-1, dtype='int')
)

ax = fig.add_subplot(gs[0,1])
for i in range(len(x_pooled)):
    ax.plot(x_pooled[i,:], alpha=0.3)
ax.plot(np.nanmean(x_pooled, axis=0), c='k')
ax.axis('off')

ax = fig.add_subplot(gs[1,0])
for i in range(y_pooled.shape[0]):
    ax.plot(-y_pooled[i,:], range(y_pooled.shape[1]), alpha=0.3)
ax.plot(-np.nanmean(y_pooled, axis=0), range(y_pooled.shape[1]), c='k')
ax.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\You\miniconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  


(-1.0505809372285675, 0.062199681799916036, -0.9, 18.9)

### plot aligned

In [6]:
aligned_ds = SplitDataset(path/"rotated")
mask_all = io.imread(path/"mask_rotated.tif", plugin="tifffile")

n_bin_x, n_bin_y = 20, 10
pooled_mx = np.full((aligned_ds.shape[1], n_bin_x*2-1, n_bin_y*2-1), np.nan)
x_pooled = np.full((aligned_ds.shape[1], n_bin_x*2-1), np.nan)
y_pooled = np.full((aligned_ds.shape[1], n_bin_y*2-1), np.nan)

fig, ax = plt.subplots(3,4, figsize=(8,6), constrained_layout=True)
fig.suptitle('rotated each')
for plane in range(aligned_ds.shape[1]):
    aligned = aligned_ds[:,plane,:,:]
    mask = mask_all[plane,:,:]
    px_mask = np.argwhere(mask>=1)
    px_str = [str(x0)+str(y0) for x0, y0 in px_mask]
    
    x_loc = np.unique(px_mask[:,0])
    y_loc = np.unique(px_mask[:,1])
    x_bins = np.digitize(x_loc, np.linspace(x_loc.min(), x_loc.max(), n_bin_x)) - 1
    y_bins = np.digitize(y_loc, np.linspace(y_loc.min(), y_loc.max(), n_bin_y)) - 1
    
    traces = np.full((aligned.shape[0], n_bin_x, n_bin_y), np.nan)
    for x in range(n_bin_x):
        for y in range(n_bin_y):
            px_here = np.array([(x0, y0) for x0 in x_loc[x_bins==x] for y0 in y_loc[y_bins==y]])
            px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
            px_here = px_here[np.isin(px_here_str, px_str)]
            if len(px_here) > 0:
                traces[:, x, y] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    traces = zscore(traces, axis=0, nan_policy='omit')
            
    mx = np.full((n_bin_x*n_bin_y, n_bin_x*2-1, n_bin_y*2-1), np.nan)
    for x0 in range(n_bin_x):
        for y0 in range(n_bin_y):
            this = traces[:, x0, y0]
            if not np.isnan(this).all():
                for x in range(n_bin_x):
                    for y in range(n_bin_y):
                        that = traces[:, x, y]
                        if not np.isnan(that).all():
                            mx[x0*n_bin_y+y0, x-x0+n_bin_x-1, y-y0+n_bin_y-1] = fast_pearson(this, that)
    mx = np.nanmean(mx, axis=0)
    pooled_mx[plane,:,:] = mx
    
    x_traces = np.full((aligned.shape[0], n_bin_x), np.nan)
    for x in range(n_bin_x):
        px_here = np.array([(x0, y0) for x0 in x_loc[x_bins==x] for y0 in y_loc])
        px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
        px_here = px_here[np.isin(px_here_str, px_str)]
        if len(px_here) > 0:
            x_traces[:, x] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    x_traces = zscore(x_traces, axis=0, nan_policy='omit')
    x_mx = np.full((n_bin_x, n_bin_x*2-1), np.nan)
    for x0 in range(n_bin_x):
        this = x_traces[:, x0]
        if not np.isnan(this).all():
            for x in range(n_bin_x):
                that = x_traces[:, x]
                if not np.isnan(that).all():
                    x_mx[x0, x-x0+n_bin_x-1] = fast_pearson(this, that)
    x_mx = np.nanmean(x_mx, axis=0)
    x_pooled[plane,:] = x_mx

    y_traces = np.full((aligned.shape[0], n_bin_y), np.nan)
    for y in range(n_bin_y):
        px_here = np.array([(x0, y0) for x0 in x_loc for y0 in y_loc[y_bins==y]])
        px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
        px_here = px_here[np.isin(px_here_str, px_str)]
        if len(px_here) > 0:
            y_traces[:, y] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    y_traces = zscore(y_traces, axis=0, nan_policy='omit')
    y_mx = np.full((n_bin_y, n_bin_y*2-1), np.nan)
    for y0 in range(n_bin_y):
        this = y_traces[:, y0]
        if not np.isnan(this).all():
            for y in range(n_bin_y):
                that = y_traces[:, y]
                if not np.isnan(that).all():
                    y_mx[y0, y-y0+n_bin_y-1] = fast_pearson(this, that)
    y_mx = np.nanmean(y_mx, axis=0)
    y_pooled[plane,:] = y_mx
    
    ax.flat[plane].matshow(mx.T, vmin=-0.6, vmax=0.6, cmap='coolwarm')
    
ax.flat[-1].matshow(np.nanmean(pooled_mx, axis=0).T, vmin=-0.6, vmax=0.6, cmap='coolwarm')
ax.flat[-1].set(title='pooled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\You\miniconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
C:\Users\You\miniconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: Mean of empty slice


[Text(0.5, 1.0, 'pooled')]

In [7]:
fig = plt.figure(figsize=(6,4), tight_layout=True)
fig.suptitle("rotated")
gs = fig.add_gridspec(2, 2, width_ratios=[3,10], height_ratios=[3,10], wspace=0, hspace=0)

ax = fig.add_subplot(gs[1,1])
ax.matshow(np.nanmean(pooled_mx, axis=0).T, vmin=-0.5, vmax=0.5, cmap='coolwarm')
ax.yaxis.set_ticks_position('right')
ax.xaxis.set_ticks_position('bottom')
ax.set(
    xticks=range(2*n_bin_x-1), xticklabels=np.linspace(-n_bin_x+1, n_bin_x-1, 2*n_bin_x-1, dtype='int'),
    yticks=range(2*n_bin_y-1), yticklabels=np.linspace(-n_bin_y+1, n_bin_y-1, 2*n_bin_y-1, dtype='int')
)

ax = fig.add_subplot(gs[0,1])
for i in range(len(x_pooled)):
    ax.plot(x_pooled[i,:], alpha=0.3)
ax.plot(np.nanmean(x_pooled, axis=0), c='k')
ax.axis('off')

ax = fig.add_subplot(gs[1,0])
for i in range(y_pooled.shape[0]):
    ax.plot(-y_pooled[i,:], range(y_pooled.shape[1]), alpha=0.3)
ax.plot(-np.nanmean(y_pooled, axis=0), range(y_pooled.shape[1]), c='k')
ax.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\You\miniconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  


(-1.0517259064463733, 0.08624403537383869, -0.9, 18.9)